## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cidreira,BR,-30.1811,-50.2056,56.52,76,100,3.85,overcast clouds
1,1,Koungou,YT,-12.7336,45.2042,80.31,69,40,19.82,scattered clouds
2,2,Agirish,RU,61.9247,63.0231,61.74,59,98,6.53,overcast clouds
3,3,Alice Springs,AU,-23.7000,133.8833,74.21,21,0,11.99,clear sky
4,4,Torbay,CA,47.6666,-52.7314,42.28,78,40,10.36,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Koungou,YT,-12.7336,45.2042,80.31,69,40,19.82,scattered clouds
3,3,Alice Springs,AU,-23.7000,133.8833,74.21,21,0,11.99,clear sky
5,5,Tautira,PF,-17.7333,-149.1500,84.22,79,94,20.71,light rain
6,6,Bengkulu,ID,-3.8004,102.2655,79.50,84,100,2.13,overcast clouds
7,7,Matara,LK,5.9485,80.5353,81.05,84,95,8.46,overcast clouds
8,8,Georgetown,MY,5.4112,100.3354,82.33,86,20,1.99,few clouds
9,9,Byron Bay,AU,-28.6500,153.6167,71.87,65,16,6.89,few clouds
10,10,Kapaa,US,22.0752,-159.3190,78.12,69,86,5.99,overcast clouds
14,14,Hithadhoo,MV,-0.6000,73.0833,84.15,69,23,8.81,few clouds
15,15,Mayo,US,38.8876,-76.5119,74.52,72,100,5.88,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                296
City                   296
Country                296
Lat                    296
Lng                    296
Max Temp               296
Humidity               296
Cloudiness             296
Wind Speed             296
Current Description    296
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Koungou,YT,80.31,scattered clouds,-12.7336,45.2042,
3,Alice Springs,AU,74.21,clear sky,-23.7000,133.8833,
5,Tautira,PF,84.22,light rain,-17.7333,-149.1500,
6,Bengkulu,ID,79.50,overcast clouds,-3.8004,102.2655,
7,Matara,LK,81.05,overcast clouds,5.9485,80.5353,
8,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
9,Byron Bay,AU,71.87,few clouds,-28.6500,153.6167,
10,Kapaa,US,78.12,overcast clouds,22.0752,-159.3190,
14,Hithadhoo,MV,84.15,few clouds,-0.6000,73.0833,
15,Mayo,US,74.52,overcast clouds,38.8876,-76.5119,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Koungou,YT,80.31,scattered clouds,-12.7336,45.2042,Villa Netibor
3,Alice Springs,AU,74.21,clear sky,-23.7000,133.8833,Desert Palms Alice Springs
5,Tautira,PF,84.22,light rain,-17.7333,-149.1500,Pension Oaoa
6,Bengkulu,ID,79.50,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
7,Matara,LK,81.05,overcast clouds,5.9485,80.5353,Amaloh Boutique Resort
8,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
9,Byron Bay,AU,71.87,few clouds,-28.6500,153.6167,Backpackers Inn on the Beach
10,Kapaa,US,78.12,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
14,Hithadhoo,MV,84.15,few clouds,-0.6000,73.0833,Scoop Guest House
15,Mayo,US,74.52,overcast clouds,38.8876,-76.5119,Resorts Resource Group International


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))